# sm-ib-2 Results

## Setup

### Global

In [1]:
# Imports

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, clear_output

In [2]:
# Global variables

results = {}

### Trace

In [3]:
# Create a path for results dictionary

def create_path(trace_type, trace_name, duration, cache_size, algorithm):
    if not trace_type in results:
        results[trace_type] = {}
    if not trace_name in results[trace_type]:
        results[trace_type][trace_name] = {}
    if not duration in results[trace_type][trace_name]:
        results[trace_type][trace_name][duration] = {}
    if not cache_size in results[trace_type][trace_name][duration]:
        results[trace_type][trace_name][duration][cache_size] = {}
    if not algorithm in results[trace_type][trace_name][duration][cache_size]:
        results[trace_type][trace_name][duration][cache_size][algorithm] = {}

In [4]:
# Read file into dictionary
# trace_collection trace_name duration cache_size algorithm 

with open('sorted.csv', 'r') as f:
    for line_orig in f:
        line = line_orig.strip().split(',')
        
        trace_type, trace_name, cache_size, real_size, algorithm, duration = line[:6]
        hits, misses, filters = map(int, line[6:9])
        promotions, demotions, private = map(int, line[9:12])
        read_hits, read_misses = map(int, line[12:14])
        write_hits, write_misses, dirty_evicts = map(int, line[14:])
        ios = hits + misses + filters
        
        if duration == '':
            duration = "full"
        
        create_path(trace_type, trace_name, duration, cache_size, algorithm)
        
        results[trace_type][trace_name][duration][cache_size][algorithm] = {
            "hits": hits,
            "read_hits": read_hits,
            "write_hits": write_hits,
            "misses": misses,
            "read_misses": read_misses,
            "write_misses": write_misses,
            "filters": filters,
            "promotions": promotions,
            "dirty_evicts": dirty_evicts,
            
            "hit-rate": 100 * (hits / ios),
            "write-rate": 100 * ((write_hits + promotions) / ios),
            "filter-rate": 100 * (filters / ios),
        }

### Interactive

In [5]:
# Setup textbox for graph title

title_text = widgets.Text(
    description='Title:',
)

In [6]:
# Setup dropdown menu to pick collection of traces for graph

workload_dropdown = widgets.Dropdown(
    options=list(results.keys()),
    description="Workload:",
)

In [7]:
# Setup dropdown menu to pick duration for graph

duration_dropdown = widgets.Dropdown(
    options=[],
    description="Duration:",
)

def setup_duration_dropdown(workload):
    workload_trace = list(results[workload].keys())[0]
    duration_dropdown.options=sorted(list(results[workload][workload_trace].keys()))
    
setup_duration_dropdown(workload_dropdown.value)

In [8]:
# Setup selection menu to pick cache size for graph

cache_size_select = widgets.SelectMultiple(
    options=['default'],
    description="Cache sizes:",
    layout=widgets.Layout(width='auto')
)

def setup_cache_size_select(workload, duration):
    workload_trace = list(results[workload].keys())[0]
    cache_size_select.options=list(results[workload][workload_trace][duration].keys())
    
setup_cache_size_select(workload_dropdown.value, duration_dropdown.value)

In [9]:
# Setup selection menu to pick algorithms for graph

algorithm_select = widgets.SelectMultiple(
    options=['default'],
    description="Algorithms:",
    layout=widgets.Layout(width='auto')
)

def setup_algorithm_select(workload, duration, cache_sizes):
    workload_trace = list(results[workload].keys())[0]
    algorithm_set = set()
    for cache_size in cache_sizes:
        for algorithm in results[workload][workload_trace][duration][cache_size]:
            algorithm_set.add(algorithm)
    
    algorithm_display_order = ('lru', 'fomo_lru', 'arc', 'fomo_arc', 'lirs', 'fomo_lirs',
                               'larc', 'marc')
    
    algorithm_options=[]
    
    for alg in algorithm_display_order:
        if alg in algorithm_set:
            algorithm_options.append(alg)
    for alg in algorithm_set:
        if alg not in algorithm_options:
            algorithm_options.append(alg)
            
    algorithm_select.options=algorithm_options
    
setup_algorithm_select(workload_dropdown.value, duration_dropdown.value, cache_size_select.value)

In [10]:
# Setup radio buttons to pick a stat to focus on for graph

stat_buttons = widgets.RadioButtons(
    options=["hit-rate", "write-rate", "filter-rate"],
    description="Stat:",
    disabled=True,
    layout=widgets.Layout(width='auto')
)

In [11]:
# Setup radio buttons to pick whether to show missing configs

missing_buttons = widgets.RadioButtons(
    options=["Show", "Hide"],
    description="Missing:",
    layout=widgets.Layout(width='auto')
)

In [12]:
# Setup radio buttons to pick whether to clear previous graphs

clear_buttons = widgets.RadioButtons(
    options=["Enable", "Disable"],
    description="Clear Graphs",
    layout=widgets.Layout(width='auto')
)

In [13]:
# Setup selection menu to pick traces for graph

trace_select = widgets.SelectMultiple(
    options=[],
    description="Traces:",
    layout=widgets.Layout(width='auto', height='100%')
)

def setup_trace_select(workload):
    trace_select.options=sorted(list(results[workload].keys()))
    
setup_trace_select(workload_dropdown.value)

In [14]:
# Setup graphing button

button = widgets.Button(
    description="Graph!",
    icon='check'
)
out = widgets.Output()

def button_clicked(_):
    if clear_buttons.value == "Enable":
        clear_output(wait=True)
        display(button)
    
    if len(cache_size_select.value) > 0 and len(algorithm_select.value) > 0:
        graphing_func()
    else:
        if len(cache_size_select.value) == 0:
            print("Please select cache size(s)")
        if len(algorithm_select.value) == 0:
            print("Please select algorithm(s)")
    #if clear_buttons.value == "Enable":
    #    display(button)
    
button.on_click(button_clicked)

In [15]:
# Setup variable to point to function for graphing

def graphing_func_default():
    pass

graphing_func = graphing_func_default

In [16]:
# Setup functions for value changes

def workload_value_changed(change):
    setup_duration_dropdown(change.new)
    setup_cache_size_select(change.new, duration_dropdown.value)
    setup_algorithm_select(change.new, duration_dropdown.value, cache_size_select.value)
    setup_trace_select(change.new)

    
def duration_value_changed(change):
    workload = workload_dropdown.value
    setup_cache_size_select(workload, change.new)
    
def cache_size_value_changed(change):
    workload = workload_dropdown.value
    duration = duration_dropdown.value
    setup_algorithm_select(workload, duration, change.new)
    
def algorithm_value_changed(change):
    stat_buttons.disabled=len(algorithm_select.value) == 0
    #graphing_func()

def stat_value_changed(change):
    pass
    #graphing_func()

In [17]:
# Setup for value changes

workload_dropdown.observe(workload_value_changed, 'value')
duration_dropdown.observe(duration_value_changed, 'value')
cache_size_select.observe(cache_size_value_changed, 'value')
algorithm_select.observe(algorithm_value_changed, 'value')
stat_buttons.observe(stat_value_changed, 'value')

### Graphing

In [18]:
# Setup graphing function

def graph_bar():
    algs = algorithm_select.value
    sizes = cache_size_select.value
    x = np.arange(len(sizes))
    fig = plt.figure(figsize=(16,6))
    
    # get indexes and y-values
    index = np.arange(len(sizes))
    bar_width = 0.9 / float(len(algs))
    opacity = 0.8
    
    # get top result dictionary
    # setup for data[algorithm][stat]
    workload_value = workload_dropdown.value
    workload_trace = list(results[workload_value].keys())[0]
    duration = duration_dropdown.value
    cache_size = list(results[workload_value][workload_trace][duration].keys())[0]
    stat = stat_buttons.value
    
    problematic_configs = set()
    
    for i, algo in enumerate(algs):
        values = []
        for size in sizes:
            size_values = []
            #for trace in results[workload_value]:
            for trace in trace_select.value:
                try:
                    size_values.append(results[workload_value][trace][duration][size][algo][stat])
                except:
                    problematic_configs.add(','.join((workload_value, trace, duration, size, algo)))
                    #raise Error
            # plot bar
            values.append(np.mean(size_values))
        plt.bar(index + (i*bar_width), values, bar_width, alpha=opacity, label=algo)

    if missing_buttons.value == "Show":
        print("\n".join(sorted(problematic_configs)))
        
    x_labels = sizes
    
    title = title_text.value
    if title == '':
        title = "{} {}".format(workload_dropdown.value, duration_dropdown.value)
    plt.title(title)
    plt.xlabel('Cache Size')
    plt.ylabel(stat)
    plt.xticks(x + ((len(algs)/2) * bar_width), x_labels)
    plt.legend(loc='upper center', bbox_to_anchor=(1.05, 0.8), ncol=1)
    
    plt.tight_layout()
    plt.show()

graphing_func = graph_bar

## Interactive Graphing

In [19]:
a_box = widgets.VBox([workload_dropdown, cache_size_select, stat_buttons])
b_box = widgets.VBox([duration_dropdown, algorithm_select, missing_buttons, clear_buttons])
c_box = widgets.VBox([trace_select])

display_box = widgets.VBox([title_text, widgets.HBox([a_box, b_box, c_box])])


display(display_box)

In [20]:
display(button)

Button(description='Graph!', icon='check', style=ButtonStyle())

In [22]:
for trace_type in results:
    for trace in results[trace_type]:
        for duration in results[trace_type][trace]:
            results_a = results[trace_type][trace][duration]
            for cache_size in results_a:
                results_b = results_a[cache_size]
                if 'larc' in results_b and 'fomo_arc' in results_b:
                    if results_b['larc']['hit-rate'] >= 1.5 * (results_b['fomo_arc']['hit-rate']):
                        print(trace_type, trace, duration, cache_size, 'larc=', results_b['larc']['hit-rate'], 'fomo_arc=', results_b['fomo_arc']['hit-rate'])

CloudPhysics w21_vscsi1.itrace 1d 0.01 larc= 1.1456477956797846 fomo_arc= 0.6056897308499343
CloudPhysics w21_vscsi1.itrace 1d 0.02 larc= 1.8518756183216194 fomo_arc= 0.6365398671175687
CloudPhysics w21_vscsi1.itrace 1d 0.05 larc= 3.9498765652137875 fomo_arc= 1.8871126286320556
CloudPhysics w65_vscsi1.itrace 1d 0.02 larc= 1.4662580509938765 fomo_arc= 0.8517044977711293
CloudPhysics w65_vscsi1.itrace 1d 0.05 larc= 2.917059631138403 fomo_arc= 1.1241903650871918
CloudPhysics w65_vscsi1.itrace 1d 0.20 larc= 10.567632391230932 fomo_arc= 5.279708145256688
CloudPhysics w94_vscsi1.itrace 1d 0.05 larc= 5.130733475683456 fomo_arc= 3.383722454650794
CloudVPS vps26356-1.blk full 0.10 larc= 20.039657174165438 fomo_arc= 11.076874910314778
FIU_1d madmax-110108-112108.1.blkparse full 0.15 larc= 23.967457466076784 fomo_arc= 15.735819551253503
MSR web_3.csv 1d 0.02 larc= 0.26780125130134674 fomo_arc= 0.12586658811163295
MSR web_3.csv 1d 0.05 larc= 0.9332873607851933 fomo_arc= 0.3283912844082764
MSR web_